In [1]:
# def save_log(file_name, log, mode='a', path = './checkpoint/'):
#     with open(path+file_name, mode) as f:
#         if mode == 'a':
#             f.write('\n')
#         if type(log) is str:
#             f.write(log)
#             print(log)
#         else:
#             log = [str(l) for l in log]
#             f.write(' '.join(log))
#             print(' '.join(log))

# save_log('test.txt', ['I like',100,'l'], 'a')

In [2]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torchaudio
from pypinyin import lazy_pinyin, Style

In [3]:
bundle = torchaudio.pipelines.WAV2VEC2_XLSR53
wave2vec_model = bundle.get_model()

In [4]:
# wave2vec_model

In [5]:
k_size = wave2vec_model.feature_extractor.conv_layers[0].conv.kernel_size[0]

In [6]:
wave2vec_model.encoder.transformer.layers[-1].final_layer_norm.normalized_shape[0]

1024

In [16]:
class ChineseStt(torch.nn.Module):
    def __init__(self, wave2vec_model, out_features):
        super(ChineseStt, self).__init__()
        self.feature_extractor = wave2vec_model.feature_extractor
        self.encoder = wave2vec_model.encoder
        in_features = wave2vec_model.encoder.transformer.layers[-1].final_layer_norm.normalized_shape[0]
        self.aux = nn.Linear(in_features, out_features, bias=True)

    def forward(self, x, lengths=None):
        x, lengths = self.feature_extractor(x, lengths)
        x = self.encoder(x, lengths)
        x = self.aux(x)
        return x, lengths

chineseStt = ChineseStt(wave2vec_model, 10)

In [19]:
chineseStt

ChineseStt(
  (feature_extractor): FeatureExtractor(
    (conv_layers): ModuleList(
      (0): ConvLayerBlock(
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
      )
      (1): ConvLayerBlock(
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
      )
      (2): ConvLayerBlock(
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
      )
      (3): ConvLayerBlock(
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
      )
      (4): ConvLayerBlock(
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
      )
      (5): ConvLayerBlock(
        (layer_

In [17]:
import requests
SPEECH_URL = "https://download.pytorch.org/torchaudio/tutorial-assets/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav"
SPEECH_FILE = "_assets/speech.wav"

if not os.path.exists(SPEECH_FILE):
    os.makedirs("_assets", exist_ok=True)
    with open(SPEECH_FILE, "wb") as file:
        file.write(requests.get(SPEECH_URL).content)

In [18]:

with torch.inference_mode():
    waveform, _ = torchaudio.load(SPEECH_FILE)
    print(waveform.shape)
    emissions, _ = chineseStt(waveform)
    emissions = torch.log_softmax(emissions, dim=-1)

emission = emissions[0].cpu().detach()

torch.Size([1, 54400])


In [26]:
from pypinyin import lazy_pinyin, Style


def chinese2pinyin(text):
    initials = lazy_pinyin(text, strict=True, style=Style.INITIALS, errors=lambda x: u'')
    finals = lazy_pinyin(text, strict=True, style=Style.FINALS, errors=lambda x: u'')
    pinyin = ''
    for i in range(len(finals)):
        pinyin+='|'
        if (initials[i] == '-'):
            continue
        pinyin+=initials[i]
        pinyin+=finals[i]
        if finals[i] == '':
            pinyin+='n'
    if pinyin[-1] == '|':
        pinyin = pinyin[:-1]
    return pinyin[1:].lower().replace('w','u')

In [41]:
import sys
importlib.reload(sys.modules['utils.dataset'])
from utils.dataset import AudioDataset, LoaderGenerator
import importlib


In [ ]:
from utils.helper import get_labels
labels = get_labels()

In [42]:
dataset = AudioDataset('./data/ST-CMDS-20170001_1-OS/')
train_set, test_set = dataset.split([8, 2])
loaderGenerator = LoaderGenerator(labels, chinese2pinyin, k_size)
train_loader = loaderGenerator.dataloader(train_set, 8)
test_loader = loaderGenerator.dataloader(test_set, 8)
for data in test_loader:
    print(data['audio'].shape, data['target'].shape)
    break

82080 20520
10260 2565
torch.Size([8, 84119]) torch.Size([8, 59])


In [43]:
from utils.textDecoder import GreedyCTCDecoder, NaiveDecoder

In [19]:
len(a), len(b)

(82080, 20520)

In [48]:
mean = lambda x: sum(x)/len(x)
mean([1,2,3])

2.0